In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from tqdm import tqdm
import lsst.daf.butler as dafButler

plt.set_loglevel('WARNING')

INFO:numexpr.utils:Note: detected 128 virtual cores but NumExpr set to maximum of 64, check "NUMEXPR_MAX_THREADS" environment variable.
INFO:numexpr.utils:Note: NumExpr detected 128 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO:numexpr.utils:NumExpr defaulting to 8 threads.


In [2]:
# Load collection
repo = '/repo/main'
instrument = "LSSTComCam"
butler= dafButler.Butler(repo, collections='u/bos/DM-48672/LSSTComCam_drp_association_v2', instrument=instrument)
butler_w04= dafButler.Butler(repo, collections='LSSTComCam/runs/DRP/DP1/w_2025_04/DM-48556', instrument=instrument)

In [ ]:
lc_refs = butler.query_datasets("goodSeeingDiff_assocDiaSrcTable")
lc_refs_04 = butler_w04.query_datasets("goodSeeingDiff_assocDiaSrcTable")

print(f"Number of tracts in diaSourceTable (current): {len(lc_refs)}")
print(f"Number of tracts in diaSourceTable (current): {len(lc_refs_04)}")


Number of tracts in diaSourceTable (current): 587
Number of tracts in diaSourceTable (current): 586


In [15]:
# Define target tracts
target_tracts = {10464, 10463, 10705, 10704}

# Filter the dataset references
filtered_refs = [ref for ref in lc_refs if ref.dataId['tract'] in target_tracts]
filtered_refs_04 = [ref for ref in lc_refs_04 if ref.dataId['tract'] in target_tracts]

In [16]:
# List of patch indices to process
patch_indices = np.arange(0, len(filtered_refs))

diaSource_list = []

# Loop through the specified tracts and fetch the data
for idx in tqdm(patch_indices):
    obj_ref = filtered_refs[idx]  # Select the corresponding obj_ref
    diaSource_patch = butler.get('goodSeeingDiff_assocDiaSrcTable', dataId=obj_ref.dataId)  # Fetch diaSource for this patch
    diaSource_list.append(diaSource_patch)

diaSource = pd.concat(diaSource_list, ignore_index=False)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 146/146 [00:08<00:00, 17.32it/s]


In [17]:
# List of patch indices to process
patch_indices = np.arange(0, len(filtered_refs_04))

diaSource_list = []

# Loop through the specified tracts and fetch the data
for idx in tqdm(patch_indices):
    obj_ref = filtered_refs_04[idx]  # Select the corresponding obj_ref
    diaSource_patch = butler_w04.get('goodSeeingDiff_assocDiaSrcTable', dataId=obj_ref.dataId)  # Fetch diaSource for this patch
    diaSource_list.append(diaSource_patch)


diaSource_04 = pd.concat(diaSource_list, ignore_index=False)


  0%|                                                                                                                                                          | 0/145 [00:00<?, ?it/s]

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 145/145 [00:08<00:00, 17.09it/s]


In [18]:
# Count the number of unique diaObjectId values for each diaSourceId
object_count_per_source = diaSource.groupby("diaSourceId")["diaObjectId"].nunique()

# Convert to DataFrame for easier handling
object_count_df = object_count_per_source.reset_index()
object_count_df.columns = ["diaSourceId", "num_unique_diaObjectId"]


# Count the number of unique diaObjectId values for each diaSourceId
object_count_per_source_04 = diaSource_04.groupby("diaSourceId")["diaObjectId"].nunique()

# Convert to DataFrame for easier handling
object_count_df_04 = object_count_per_source_04.reset_index()
object_count_df_04.columns = ["diaSourceId", "num_unique_diaObjectId"]


In [19]:
# diaSources that are associated with more than one diaObject in the new version
object_count_df[object_count_df['num_unique_diaObjectId']>1]

,diaSourceId,num_unique_diaObjectId
160357,191543844066557962,2
160358,191543844066557963,2


In [20]:
# diaSources that are associated with more than one diaObject in the weekly 04
object_count_df_04[object_count_df_04['num_unique_diaObjectId']>1]

,diaSourceId,num_unique_diaObjectId
76,191403211796512845,3
80,191403211796512849,3
84,191403211796512853,2
85,191403211796512854,2
86,191403211796512855,2
...,...,...
245009,191895765117829242,2
245011,191895765117829244,2
245012,191895765117829245,2
245017,191895765117829250,2
